<a href="https://colab.research.google.com/github/ary3120-droid/myproject/blob/main/MERK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**실행**

In [ ]:
import pandas as pd
df = pd.read_csv('/content/MERU/train.tsv', sep='\t')
print(df.head())

   train_id                                 name  item_condition_id  \
0         0  MLB Cincinnati Reds T Shirt Size XL                  3   
1         1     Razer BlackWidow Chroma Keyboard                  3   
2         2                       AVA-VIV Blouse                  1   
3         3                Leather Horse Statues                  1   
4         4                 24K GOLD plated rose                  1   

                                       category_name brand_name  price  \
0                                  Men/Tops/T-shirts        NaN   10.0   
1  Electronics/Computers & Tablets/Components & P...      Razer   52.0   
2                        Women/Tops & Blouses/Blouse     Target   10.0   
3                 Home/Home Décor/Home Décor Accents        NaN   35.0   
4                            Women/Jewelry/Necklaces        NaN   44.0   

   shipping                                   item_description  
0       1.0                                 No description yet 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head(20)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1.0,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0.0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1.0,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1.0,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0.0,Complete with certificate of authenticity
5,5,Bundled items requested for Ruie,3,Women/Other/Other,NaN,59.0,0.0,"Banana republic bottoms, Candies skirt with ma..."
6,6,Acacia pacific tides santorini top,3,Women/Swimwear/Two-Piece,Acacia Swimwear,64.0,0.0,Size small but straps slightly shortened to fi...
7,7,Girls cheer and tumbling bundle of 7,3,Sports & Outdoors/Apparel/Girls,Soffe,6.0,1.0,You get three pairs of Sophie cheer shorts siz...
8,8,Girls Nike Pro shorts,3,Sports & Outdoors/Apparel/Girls,Nike,19.0,0.0,Girls Size small Plus green. Three shorts total.
9,9,Porcelain clown doll checker pants VTG,3,Vintage & Collectibles/Collectibles/Doll,NaN,8.0,0.0,I realized his pants are on backwards after th...


데이터 정제 (name 컬럼)

-사이즈 정보는 불필요하다 생각

-특수 문자 제거

-공백 정리

In [ ]:
import re
import pandas as pd

def clean_name_remove_size(text):
    if pd.isna(text):
        return ""

    # 소문자
    text = str(text).lower()

    # (1) 사이즈 알파벳 제거
    size_words = [
        "xxs", "xs", "s", "m", "l", "xl", "xxl", "xxxl",
        "2xl", "3xl", "4xl", "size"
    ]
    for w in size_words:
        text = re.sub(rf"\b{w}\b", " ", text)

    # (2) 용량 단위 제거 (숫자 + ml, g, kg, l 등)
    text = re.sub(r"\b\d+\s*(ml|g|kg|l)\b", " ", text)

    # (3) 숫자 사이즈 제거 (1~3자리 숫자 단독)
    text = re.sub(r"\b\d{1,3}\b", " ", text)

    # (4) 특수문자 모두 제거 (영문/숫자 외 > 공백)
    text = re.sub(r"[^a-z0-9]+", " ", text)

    # (5) 공백 정리
    text = re.sub(r"\s+", " ", text).strip()

    return text


def apply_clean_name(df):
    df["name_clean"] = df["name"].fillna("").apply(clean_name_remove_size)
    return df


확인

In [ ]:
df = apply_clean_name(df)

df[["name", "name_clean"]].head(20)

,name,name_clean
0,MLB Cincinnati Reds T Shirt Size XL,mlb cincinnati reds t shirt
1,Razer BlackWidow Chroma Keyboard,razer blackwidow chroma keyboard
2,AVA-VIV Blouse,ava viv blouse
3,Leather Horse Statues,leather horse statues
4,24K GOLD plated rose,24k gold plated rose
5,Bundled items requested for Ruie,bundled items requested for ruie
6,Acacia pacific tides santorini top,acacia pacific tides santorini top
7,Girls cheer and tumbling bundle of 7,girls cheer and tumbling bundle of
8,Girls Nike Pro shorts,girls nike pro shorts
9,Porcelain clown doll checker pants VTG,porcelain clown doll checker pants vtg


브랜드 컬럼의 결측치

name에서 브랜드 자동 탐지 → 결측 채우기


In [ ]:
def build_brand_dict(df):
    brands = (
        df['brand_name']
        .dropna()
        .str.lower()
        .unique()
        .tolist()
    )
    # 길이 필터 없음 > 전체 사용
    return set(brands)

def fill_brand_from_name(df, name_col="name_clean", brand_col="brand_name"):
    brand_dict = build_brand_dict(df)

    new_brand = []
    for name, brand in zip(df[name_col], df[brand_col]):
        # 원래 brand_name 있으면 그대로 사용
        if isinstance(brand, str) and brand.strip() != "":
            new_brand.append(brand)
            continue

        # 결측일 때만 > name에서 브랜드 찾기
        detected = ""
        for b in brand_dict:
            if b in name:   # 부분 포함하면 채움
                detected = b
                break

        new_brand.append(detected)

    df['brand_name_filled'] = new_brand
    return df

In [ ]:
df = apply_clean_name(df)   # 아까 만든 name_clean 함수

# name > brand_name 채우기
df = fill_brand_from_name(df)


In [ ]:
df = apply_clean_name(df)     # name_clean 만들기
df = fill_brand_from_name(df) # 브랜드 자동 채우기

df[['name_clean','brand_name','brand_name_filled']].head(20)


,name_clean,brand_name,brand_name_filled
0,mlb cincinnati reds t shirt,NaN,inc
1,razer blackwidow chroma keyboard,Razer,Razer
2,ava viv blouse,Target,Target
3,leather horse statues,NaN,
4,24k gold plated rose,NaN,
5,bundled items requested for ruie,NaN,m
6,acacia pacific tides santorini top,Acacia Swimwear,Acacia Swimwear
7,girls cheer and tumbling bundle of,Soffe,Soffe
8,girls nike pro shorts,Nike,Nike
9,porcelain clown doll checker pants vtg,NaN,


정제된 컬럼(name_clean, brand_name_filled)만 남긴다.

>>name_clean	/brand

In [ ]:
df = df.drop(columns=["name", "brand_name"], errors="ignore")

In [ ]:
df = df.rename(columns={"brand_name_filled": "brand"})
df = df.drop(columns=["name", "brand_name"], errors="ignore")

In [ ]:
# category 결측
df["category_name"] = df["category_name"].fillna("unknown")

# description 결측
df["item_description"] = df["item_description"].fillna("")

# brand_name_filled을 brand로 승격
df["brand"] = df["brand"].replace("", "unknown")

# 최종 brand NaN 채우기
df["brand"] = df["brand"].fillna("unknown")

# price 이상치 제거 예시
df = df[df["price"] > 0]     # 0원 상품 제거


In [ ]:
df.head(10)

,train_id,item_condition_id,category_name,price,shipping,item_description,name_clean,brand
0,0,3,Men/Tops/T-shirts,10.0,1.0,No description yet,mlb cincinnati reds t shirt,inc
1,1,3,Electronics/Computers & Tablets/Components & P...,52.0,0.0,This keyboard is in great condition and works ...,razer blackwidow chroma keyboard,Razer
2,2,1,Women/Tops & Blouses/Blouse,10.0,1.0,Adorable top with a hint of lace and a key hol...,ava viv blouse,Target
3,3,1,Home/Home Décor/Home Décor Accents,35.0,1.0,New with tags. Leather horses. Retail for [rm]...,leather horse statues,unknown
4,4,1,Women/Jewelry/Necklaces,44.0,0.0,Complete with certificate of authenticity,24k gold plated rose,unknown
5,5,3,Women/Other/Other,59.0,0.0,"Banana republic bottoms, Candies skirt with ma...",bundled items requested for ruie,m
6,6,3,Women/Swimwear/Two-Piece,64.0,0.0,Size small but straps slightly shortened to fi...,acacia pacific tides santorini top,Acacia Swimwear
7,7,3,Sports & Outdoors/Apparel/Girls,6.0,1.0,You get three pairs of Sophie cheer shorts siz...,girls cheer and tumbling bundle of,Soffe
8,8,3,Sports & Outdoors/Apparel/Girls,19.0,0.0,Girls Size small Plus green. Three shorts total.,girls nike pro shorts,Nike
9,9,3,Vintage & Collectibles/Collectibles/Doll,8.0,0.0,I realized his pants are on backwards after th...,porcelain clown doll checker pants vtg,unknown
